# Pareto Q-Learning en `mo-lunar-lander-v3`

Este notebook implementa el algoritmo de Pareto Q-Learning sobre el entorno multiobjetivo `mo-lunar-lander-v3` usando Python y `mo-gymnasium`.

**Abreviaturas utilizadas**:
- `acc`: acumulada
- `est`: estados
- `eps`: episodios
- `obj`: objetivos
- `act`: acciones


In [ ]:

import numpy as np
import gymnasium as gym
import mo_gymnasium as mo_gym
from collections import defaultdict
import matplotlib.pyplot as plt


In [ ]:

entorno = mo_gym.make("mo-lunar-lander-v3")
num_act = entorno.action_space.n
num_est = 1000
alpha = 0.1
gamma = 0.99
epsilon = 0.1
eps = 500
num_obj = len(entorno.reward_range)

Q = defaultdict(lambda: np.zeros((num_act, num_obj)))
frontera_pareto = []


In [ ]:

def discretizar(estado):
    return tuple(np.round(estado, decimals=1))

def seleccionar_accion(estado):
    if np.random.rand() < epsilon:
        return np.random.randint(num_act)
    else:
        valores = Q[estado]
        escalarizados = valores @ np.ones(num_obj)
        return np.argmax(escalarizados)

def actualizar_pareto(frontera, nuevo_vec):
    no_dominados = []
    for vec in frontera:
        if np.all(vec >= nuevo_vec) and np.any(vec > nuevo_vec):
            return frontera
        elif not (np.all(nuevo_vec >= vec) and np.any(nuevo_vec > vec)):
            no_dominados.append(vec)
    no_dominados.append(nuevo_vec)
    return no_dominados


In [ ]:

recompensas_hist = []

for ep in range(eps):
    estado, _ = entorno.reset()
    estado = discretizar(estado)
    terminado = False
    recompensa_acc = np.zeros(num_obj)

    while not terminado:
        accion = seleccionar_accion(estado)
        nuevo_estado, recompensa_vec, fin, truncado, _ = entorno.step(accion)
        nuevo_estado = discretizar(nuevo_estado)
        terminado = fin or truncado

        mejor_sig = np.max(Q[nuevo_estado], axis=0)
        Q[estado][accion] = (1 - alpha) * Q[estado][accion] + alpha * (recompensa_vec + gamma * mejor_sig)

        estado = nuevo_estado
        recompensa_acc += recompensa_vec

    recompensas_hist.append(recompensa_acc)
    frontera_pareto = actualizar_pareto(frontera_pareto, recompensa_acc)

    if ep % 50 == 0:
        print(f"Episodio {ep}, Recompensa acumulada: {recompensa_acc}")

entorno.close()


In [ ]:

recompensas_arr = np.array(recompensas_hist)
plt.plot(recompensas_arr[:, 0], label='Recompensa 1')
plt.plot(recompensas_arr[:, 1], label='Recompensa 2')
plt.title("Recompensas por episodio")
plt.xlabel("Episodio")
plt.ylabel("Recompensas")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

frontera_pareto = np.array(frontera_pareto)
plt.scatter(frontera_pareto[:, 0], frontera_pareto[:, 1], color='red')
plt.title("Frontera de Pareto aproximada")
plt.xlabel("Objetivo 1")
plt.ylabel("Objetivo 2")
plt.grid(True)
plt.tight_layout()
plt.show()
